# Kolmogorov-Smirnov data drift detector on CIFAR-10

### Method

The drift detector applies feature-wise two-sample [Kolmogorov-Smirnov](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) (K-S) tests. For multivariate data, the obtained p-values for each feature are aggregated either via the [Bonferroni](https://mathworld.wolfram.com/BonferroniCorrection.html) or the [False Discovery Rate](http://www.math.tau.ac.il/~ybenja/MyPapers/benjamini_hochberg1995.pdf) (FDR) correction. The Bonferroni correction is more conservative and controls for the probability of at least one false positive. The FDR correction on the other hand allows for an expected fraction of false positives to occur.

For high-dimensional data, we typically want to reduce the dimensionality before computing the feature-wise univariate K-S tests and aggregating those via the chosen correction method. Following suggestions in [Failing Loudly: An Empirical Study of Methods for Detecting Dataset Shift](https://arxiv.org/abs/1810.11953), we incorporate Untrained AutoEncoders (UAE), black-box shift detection using the classifier's softmax outputs ([BBSDs](https://arxiv.org/abs/1802.03916)) and [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) as out-of-the box preprocessing methods. Preprocessing methods which do not rely on the classifier will usually pick up drift in the input data, while BBSDs focuses on label shift.

### Dataset

[CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) consists of 60,000 32 by 32 RGB images equally distributed over 10 classes. We evaluate the drift detector on the CIFAR-10-C dataset ([Hendrycks & Dietterich, 2019](https://arxiv.org/abs/1903.12261)). The instances in
CIFAR-10-C have been corrupted and perturbed by various types of noise, blur, brightness etc. at different levels of severity, leading to a gradual decline in the classification model performance. We also check for drift against the original test set with class imbalances. 

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, InputLayer, Reshape

from alibi_detect.cd import KSDrift
from alibi_detect.cd.preprocess import uae, hidden_output
from alibi_detect.models.resnet import scale_by_instance
from alibi_detect.utils.fetching import fetch_tf_model
from alibi_detect.utils.saving import save_detector, load_detector
from alibi_detect.datasets import fetch_cifar10c, corruption_types_cifar10c

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


### Load data

Original CIFAR-10 data:

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = y_train.astype('int64').reshape(-1,)
y_test = y_test.astype('int64').reshape(-1,)

For CIFAR-10-C, we can select from the following corruption types at 5 severity levels:

In [ ]:
corruptions = corruption_types_cifar10c()
print(corruptions)

Let's pick a subset of the corruptions at corruption level 5. Each corruption type consists of perturbations on all of the original test set images.

In [ ]:
corruption = ['gaussian_noise', 'motion_blur', 'brightness', 'pixelate']
X_corr, y_corr = fetch_cifar10c(corruption=corruption, severity=5, return_X_y=True)
X_corr = X_corr.astype('float32') / 255

We split the original test set in a reference dataset and a dataset which should not be rejected under the *H<sub>0</sub>* of the K-S test. We also split the corrupted data by corruption type:

In [ ]:
np.random.seed(0)
n_test = X_test.shape[0]
idx = np.random.choice(n_test, size=n_test // 2, replace=False)
idx_h0 = np.delete(np.arange(n_test), idx, axis=0)
X_ref,y_ref = X_test[idx], y_test[idx]
X_h0, y_h0 = X_test[idx_h0], y_test[idx_h0]
print(X_ref.shape, X_h0.shape)

In [ ]:
# check that the classes are more or less balanced
classes, counts_ref = np.unique(y_ref, return_counts=True)
counts_h0 = np.unique(y_h0, return_counts=True)[1]
print('Class Ref H0')
for cl, cref, ch0 in zip(classes, counts_ref, counts_h0):
    assert cref + ch0 == n_test // 10
    print('{}     {} {}'.format(cl, cref, ch0))

In [ ]:
X_c = []
n_corr = len(corruption)
for i in range(n_corr):
    X_c.append(X_corr[i * n_test:(i + 1) * n_test])

We can visualise the same instance for each corruption type:

In [ ]:
i = 1

n_test = X_test.shape[0]
plt.title('Original')
plt.axis('off')
plt.imshow(X_test[i])
plt.show()
for _ in range(len(corruption)):
    plt.title(corruption[_])
    plt.axis('off')
    plt.imshow(X_corr[n_test * _+ i])
    plt.show()

We can also verify that the performance of a classification model on CIFAR-10 drops significantly on this perturbed dataset:

In [ ]:
clf = fetch_tf_model('cifar10', 'resnet32')
acc = clf.evaluate(scale_by_instance(X_test), y_test, batch_size=128, verbose=0)[1]
print('Test set accuracy:')
print('Original {:.4f}'.format(acc))
for _ in range(len(corruption)):
    acc = clf.evaluate(scale_by_instance(X_c[_]), y_test, batch_size=128, verbose=0)[1]
    print('{} {:.4f}'.format(corruption[_], acc))

Given the drop in performance, it is important that we detect the harmful data drift!

### Detect drift

We are trying to detect data drift on high-dimensional (*32x32x3*) data using an aggregation of univariate K-S tests. It therefore makes sense to apply dimensionality reduction first. Some dimensionality reduction methods also used in [Failing Loudly: An Empirical Study of Methods for Detecting Dataset Shift](https://arxiv.org/pdf/1810.11953.pdf) are readily available: **UAE** (Untrained AutoEncoder), **BBSDs** (black-box shift detection using the classifier's softmax outputs) and **PCA**. 

#### Untrained AutoEncoder

First we try UAE:

In [ ]:
# define encoder
encoding_dim = 32
encoder_net = tf.keras.Sequential(
  [
      InputLayer(input_shape=(32, 32, 3)),
      Conv2D(64, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(128, 4, strides=2, padding='same', activation=tf.nn.relu),
      Conv2D(512, 4, strides=2, padding='same', activation=tf.nn.relu),
      Flatten(),
      Dense(encoding_dim,)
  ]
)

# initialise drift detector
p_val = .05
cd = KSDrift(
    p_val=p_val,        # p-value for K-S test 
    X_ref=X_ref,       # test against original test set
    preprocess_fn=uae,  # UAE for dimensionality reduction
    preprocess_kwargs={'encoder_net': encoder_net, 'batch_size': 128},
    alternative='two-sided'  # other options: 'less', 'greater'
)

# we can also save/load an initialised detector
filepath = 'my_path'  # change to directory where detector is saved
save_detector(cd, filepath)
cd = load_detector(filepath)

The p-value used by the detector for the multivariate data with *encoding_dim* features is equal to *p_val / encoding_dim* because of the [Bonferroni correction](https://mathworld.wolfram.com/BonferroniCorrection.html).

In [ ]:
assert cd.p_val / cd.n_features == p_val / encoding_dim

Let's check whether the detector thinks drift occurred within the original test set:

In [ ]:
preds_h0 = cd.predict(X_h0, return_feature_score=True)
labels = ['No!', 'Yes!']
print('Drift? {}'.format(labels[preds_h0['data']['is_drift'].item()]))

As expected, no drift occurred. We can also inspect the p-values for each univariate K-S test by (encoded) feature before the multivariate correction. Most of them are well above the $0.05$ threshold:

In [ ]:
print(preds_h0['data']['feature_score'])

Let's now check the predictions on the perturbed data:

In [ ]:
for x, c in zip(X_c, corruption):
    preds = cd.predict(x, return_feature_score=True)
    print(f'Corruption type: {c}')
    print('Drift? {}'.format(labels[preds['data']['is_drift'].item()]))
    print('Feature-wise p-values:')
    print(preds['data']['feature_score'])
    print('')

#### BBSDs

For **BBSDs**, we use the classifier's softmax outputs for black-box shift detection. This method is based on [Detecting and Correcting for Label Shift with Black Box Predictors](https://arxiv.org/abs/1802.03916). The ResNet classifier is trained on data standardised by instance so we need to rescale the data.

In [ ]:
X_train = scale_by_instance(X_train)
X_test = scale_by_instance(X_test)
for i in range(n_corr):
    X_c[i] = scale_by_instance(X_c[i])
X_ref = scale_by_instance(X_ref)
X_h0 = scale_by_instance(X_h0)

Initialisation of the drift detector. Here we use the output of the softmax layer to detect the drift, but other hidden layers can be extracted as well by setting *'layer'* to the index of the desired hidden layer in the model:

In [ ]:
p_val = .05
cd = KSDrift(
    p_val=p_val,
    X_ref=X_ref,
    preprocess_fn=hidden_output,
    preprocess_kwargs={'model': clf, 'layer': -1, 'batch_size': 128},  # use output softmax layer
)

Again we can see that the p-value used by the detector for the multivariate data with 10 features (number of CIFAR-10 classes) is equal to *p_val / 10* because of the [Bonferroni correction](https://mathworld.wolfram.com/BonferroniCorrection.html).

In [ ]:
assert cd.p_val / cd.n_features == p_val / 10

There is no drift on the original held out test set:

In [ ]:
preds_h0 = cd.predict(X_h0, return_feature_score=True)
print('Drift? {}'.format(labels[preds_h0['data']['is_drift'].item()]))
print(preds_h0['data']['feature_score'])

We compare this with the perturbed data:

In [ ]:
for x, c in zip(X_c, corruption):
    preds = cd.predict(x, return_feature_score=True)
    print(f'Corruption type: {c}')
    print('Drift? {}'.format(labels[preds['data']['is_drift'].item()]))
    print('Feature-wise p-values:')
    print(preds['data']['feature_score'])
    print('')

### Label drift

We could also use only the original test set data but with class imbalances between the reference data *X_ref* and the tested data *X_h1*. The reference data will use $75$% of the instances of the first 5 classes and only $25$% of the last 5. The data used for drift testing then uses respectively $25$% and $75$% of the test instances for the first and last 5 classes.

In [ ]:
np.random.seed(0)
# get index for each class in the test set
num_classes = len(np.unique(y_test))
idx_by_class = [np.where(y_test == c)[0] for c in range(num_classes)]
# sample imbalanced data for different classes for X_ref and X_h1
perc_ref = .75
perc_ref_by_class = [perc_ref if c < 5 else 1 - perc_ref for c in range(num_classes)]
n_by_class = n_test // num_classes
X_ref = []
X_h1 = []
for _ in range(num_classes):
    idx_class_ref = np.random.choice(n_by_class, size=int(perc_ref_by_class[_] * n_by_class), replace=False)
    idx_ref = idx_by_class[_][idx_class_ref]
    idx_class_h1 = np.delete(np.arange(n_by_class), idx_class_ref, axis=0)
    idx_h1 = idx_by_class[_][idx_class_h1]
    assert idx_ref != idx_h1
    X_ref.append(X_test[idx_ref])
    X_h1.append(X_test[idx_h1])
X_ref = np.concatenate(X_ref)
X_h1 = np.concatenate(X_h1)
print(X_ref.shape, X_h1.shape)

Update reference dataset for the detector and make predictions:

In [ ]:
cd.X_ref = X_ref

In [ ]:
preds_h1 = cd.predict(X_h1, return_feature_score=True)
print('Drift? {}'.format(labels[preds_h1['data']['is_drift'].item()]))
print(preds_h1['data']['feature_score'])

### Update reference data

So far we have kept the reference data the same throughout the experiments. It is possible however that we want to test a new batch against the last *N* instances or against a batch of instances of fixed size where we give each instance we have seen up until now the same chance of being in the reference batch (*reservoir sampling*). The `update_X_ref` argument allows you to change the reference data update rule. It is a Dict which takes as key the update rule (*'last'* for last *N* instances or *'reservoir_sampling'*) and as value the batch size *N* of the reference data. You can also save the detector after the prediction calls to save the updated reference data.

In [ ]:
N = 7500
cd = KSDrift(
    p_val=.05,
    X_ref=X_ref,
    update_X_ref={'reservoir_sampling': N},
    preprocess_fn=hidden_output,
    preprocess_kwargs={'model': clf, 'layer': -1, 'batch_size': 128},
)

The reference data is now updated with each `predict` call. Say we start with our imbalanced reference set and make a prediction on the remaining test set data *X_h1*, then the drift detector will figure out data drift has occurred.

In [ ]:
preds_h1 = cd.predict(X_h1)
print('Drift? {}'.format(labels[preds_h1['data']['is_drift'].item()]))

We can now see that the reference data consists of *N* instances, obtained through reservoir sampling.

In [ ]:
assert cd.X_ref.shape[0] == N

We then draw a random sample from the training set and compare it with the updated reference data. This still highlights that there is data drift but will update the reference data again:

In [ ]:
np.random.seed(0)
perc_train = .5
n_train = X_train.shape[0]
idx_train = np.random.choice(n_train, size=int(perc_train * n_train), replace=False)

In [ ]:
preds_train = cd.predict(X_train[idx_train])
print('Drift? {}'.format(labels[preds_train['data']['is_drift'].item()]))

When we draw a new sample from the training set, it highlights that it is not drifting anymore against the reservoir in *X_ref*.

In [ ]:
perc_train = .1
idx_train = np.random.choice(n_train, size=int(perc_train * n_train), replace=False)
preds_train = cd.predict(X_train[idx_train])
print('Drift? {}'.format(labels[preds_train['data']['is_drift'].item()]))

### Multivariate correction mechanism

Instead of the Bonferroni correction for multivariate data, we can also use the less conservative [False Discovery Rate](http://www.math.tau.ac.il/~ybenja/MyPapers/benjamini_hochberg1995.pdf) (FDR) correction. See [here](https://riffyn.com/riffyn-blog/2017/10/29/false-discovery-rate) or [here](https://matthew-brett.github.io/teaching/fdr.html) for nice explanations. While the Bonferroni correction controls the probability of at least one false positive, the FDR correction controls for an expected amount of false positives. The `p_val` argument at initialisation time can be interpreted as the acceptable q-value when the FDR correction is applied.

In [ ]:
cd = KSDrift(
    p_val=.05,
    correction='fdr',
    X_ref=X_ref,
    preprocess_fn=hidden_output,
    preprocess_kwargs={'model': clf, 'layer': -1, 'batch_size': 128},  # use output softmax layer
)

preds_h1 = cd.predict(X_h1)
print('Drift? {}'.format(labels[preds_h1['data']['is_drift'].item()]))

### Train vs. test set drift

Let us do a last check and see whether there is data drift between the original train and test sets. We will use both the **BBSDs** and **Untrained AutoEncoder** preprocessing techniques. We start with BBSDs:

In [ ]:
cd = KSDrift(
    p_val=.05,
    X_ref=X_train,
    preprocess_fn=hidden_output,
    preprocess_kwargs={'model': clf, 'layer': -1, 'batch_size': 128},
)

preds_test = cd.predict(X_test)
print('Drift? {}'.format(labels[preds_test['data']['is_drift'].item()]))

So when we use the softmax output of the classification model, we pick up drift. It is important to notice that this is drift with respect to the classifier output. This means that the detector picked up a difference between the distributions of the softmax probabilities of the model on the train and test sets.

When we use the UAE to preprocess the data we do not care about the distribution of the labels being predicted and only take random projections of the input on the latent dimension into account.

In [ ]:
cd = KSDrift(
    p_val=.05,
    X_ref=X_train,
    preprocess_fn=uae,
    preprocess_kwargs={'encoder_net': encoder_net, 'batch_size': 128},
)

preds_test = cd.predict(X_test)
print('Drift? {}'.format(labels[preds_test['data']['is_drift'].item()]))

The UAE detector does not pick up drift. This means that the input data of the classification model does not seem to drift between the train and test set, but the model output does!